#### ĐỌC CÁC THÔNG SỐ ĐẦU VÀO

In [4]:
from math import sqrt
# Lấy số đỉnh, số cạnh, số luồng, đỉnh nguồn, đỉnh đích
f = open("F:\Lập trình\Python\PY.txt","r")
read = f.readline()
read=read.split()
vertex = int(read[0])
edges = int(read[1])
flow = int(read[2])
source = int(read[3])
sink = int(read[4])

# Tọa độ các node
note_position = {}
for i in range (vertex):
    loc = (f.readline()).split(' ')
    note_position[i] = (int(loc[0]),int(loc[1]))

# Đồ thị
graph={}
list_save_edges = []
for i in range (vertex):
    graph[i]={}
for i in f:
    line = i.split()
    list_save_edges.append(line)
    dict_temp = {int(line[1]): (int(line[2]),int(line[3]))}
    graph[int(line[0])].update(dict_temp)
graph[vertex] = {source: (flow,0)}
graph[vertex+1] = {}
graph[sink].update({vertex+1 : (flow,0)})

# Đỉnh giả
source = vertex
sink = vertex+1

#### HEURISTIC TÍNH KHOẢNG CÁCH TRỰC TIẾP TỪ CÁC NODE KHÁC ĐẾN NODE ĐÍCH

In [5]:
air = []
for i in range(vertex):
    kc = f'{sqrt((note_position[i][0] - note_position[vertex-1][0])**2 + (note_position[i][1] - note_position[vertex-1][1])**2):.2f}'
    air.append(float(kc))
air.append(10000)
air.append(0)

#### ÁP DỤNG GIẢI BÀI TOÁN MINCOST FLOW

In [6]:
import heapq

list_ket_qua = []

def min_cost_flow(graph, source, sink, flow):
    # KHỞI TẠO TỔNG LUỒNG VÀ TỔNG CHI PHÍ
    total_cost = 0
    total_flow = 0
    print('Kết quả bài toán:')
    while flow > 0:
        # DÙNG THUẬT TOÁN GBFS ĐỂ TÌM ĐƯỜNG ĐI NGẮN NHẤT TỪ SOURCE ĐẾN SINK
        prev = {} # LƯU ĐƯỜNG ĐI
        heap = [(air[source], source)] # HÀNG ĐỢI ƯU TIÊN ĐỂ LẤY ĐỈNH GẦN NODE ĐÍCH NHẤT RA TRƯỚC
        visited =[0] * (vertex + 2) # DÙNG ĐỂ ĐÁNH DẤU ĐÃ GHÉ ĐỈNH [I] HAY CHƯA
        while heap:
            d, node = heapq.heappop(heap)
            if visited[node] == 1: continue
            visited[node] = 1
            for neighbor in graph[node]:
                if visited[neighbor] == 1:continue
                if graph[node][neighbor][0] > 0:
                    prev[neighbor] = (node, graph[node][neighbor][0], graph[node][neighbor][1])
                    heapq.heappush(heap, (air[neighbor], neighbor))
        if visited[sink] == 0:
            # KHÔNG CÒN ĐƯỜNG ĐI TỪ SOURCE ĐẾN SINK
            break
        print('----------------')

        # TÌM FLOW NHỎ NHẤT TỪ TRÊN ĐƯỜNG VỪA TÌM ĐƯỢC
        node = sink
        min_flow = float('inf')
        path=[]
        colored_edges_1 = []
        while (node != source):
            min_flow = min(min_flow, prev[node][1])
            node = prev[node][0]
            if node == source: break
            path.append(str(node)) # LƯU ĐƯỜNG ĐI VỪA TÌM ĐƯỢC
        path.reverse()
        print('-->'.join(path))

        # CẬP NHẬT LẠI FLOW CÒN LẠI, TÍNH TỔNG FLOW ĐÃ CHUYỂN
        flow -= min_flow
        total_flow+= min_flow
        print('Max Flow:',min_flow)
       # CẬP NHẬT ĐỒ THỊ
        node = sink
        cost_path = 0
        while (node != source):
            cost_path += min_flow * prev[node][2]
            total_cost += min_flow * prev[node][2]
            prev_node, _, _ = prev[node]
            temp = graph[prev_node][node]
            graph[prev_node][node] = (temp[0] - min_flow, temp[1])
            node = prev_node
        print(f'Chi phí đường đi: {cost_path}$')
        #### GIAO DIỆN
        # LƯU CÁC CẠNH ĐI QUA
        for i in range(len(path)):
              if i <= len(path) - 2:
                colored_edges_1.append((int(path[i]),int(path[i+1])))
        list_ket_qua.append(['----------------','-->'.join(path),f'Max Flow:{min_flow}',f'Chi phí đường đi: {cost_path}$'])
        # LƯU CÁC ĐỈNH ĐI QUA
        list_duong_di.append(path)
        colored_edges.append(colored_edges_1)
        list_minflow.append(min_flow)
    return total_flow, total_cost
list_minflow = []
list_duong_di = []
colored_edges = []

total_flow, total_cost = min_cost_flow(graph, source, sink, flow)
if total_flow == 0:
      print('Không thể tìm đường để vận chuyển từ {} đến {}'.format(0,vertex-1))
elif total_flow == flow:
      print('Có thể chuyển hết hàng từ {} đến {}'.format(0,vertex-1))
else:
      print('Có thể chuyển 1 phần hàng từ {} đến {} và còn {} đơn vị hàng chưa chuyển'.format(0,vertex-1,flow-total_flow))
print("Total Flow:", total_flow)
print("Total Cost:", total_cost)


Kết quả bài toán:
----------------
0-->2-->3-->4
Max Flow: 8
Chi phí đường đi: 56$
----------------
0-->1-->3-->4
Max Flow: 4
Chi phí đường đi: 32$
Có thể chuyển hết hàng từ 0 đến 4
Total Flow: 12
Total Cost: 88


In [7]:
import tkinter as tk
import networkx as nx
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt

# Hàm tạo hình ảnh từ NetworkX đồ thị
dem = 0
dem_lap_lai = 0
total_minflow = 0
count_repeat = 0
toc_do = 2000
list_copy = []
list_lables = []
H = nx.DiGraph()
animation_running = False
def create_networkx_image(graph):
    fig, ax = plt.subplots()
    # làm sao để tổng quát hóa vị trí các đỉnh
    # Hiệu ứng nhấp nháy
    global dem 
    global dem_lap_lai
    global total_minflow
    global count_repeat
    nx.draw(graph, note_position, ax=ax, with_labels=True,connectionstyle='arc3,rad=0.1') # vẽ đồ thị
    red_edges_on = True
    if len(list_duong_di) != 0:
        if red_edges_on:
            if dem >= len(list_duong_di[dem_lap_lai]):
                dem = 0
                dem_lap_lai += 1
            # đổi màu nút đang đi thành đỏ
            nx.draw(graph, note_position, ax=ax, with_labels=True,connectionstyle='arc3,rad=0.1',node_color=['red' if node == int(list_duong_di[dem_lap_lai][dem]) else 'skyblue' for node in graph.nodes()])
            if dem == 0:
                list_copy.clear()
                # H.clear()
                list_lables.clear()
                nx.draw_networkx_edges(G, pos=note_position,ax=ax, edge_color='black',connectionstyle='arc3,rad=0.1')  # Tô màu cho các cạnh cụ thể
                labels = nx.get_edge_attributes(G, 'weight')  # Trọng số của các cạnh
                nx.draw_networkx_edge_labels(G, note_position,ax=ax, edge_labels=labels,connectionstyle='arc3,rad=0.1')  # Vẽ trọng số
                labels = nx.get_edge_attributes(H, 'weight')  # Trọng số của các cạnh
                nx.draw_networkx_edge_labels(H, note_position,ax=ax, edge_labels=labels,connectionstyle='arc3,rad=0.1')  # Vẽ trọng số
            else:
                for i in range(len(list_save_edges)):
                    if int(list_save_edges[i][0]) == colored_edges[dem_lap_lai][(dem-1)][0] and int(list_save_edges[i][1]) == colored_edges[dem_lap_lai][(dem-1)][1]:
                        list_lables.append([list_save_edges[i][2],list_save_edges[i][3]])
                count_repeat = 0
                for i in range(len(colored_edges)):
                    count_repeat += colored_edges[i].count((colored_edges[dem_lap_lai][(dem-1)][0],colored_edges[dem_lap_lai][(dem-1)][1]))
                positions = [index for index, value in enumerate(colored_edges) if (colored_edges[dem_lap_lai][(dem-1)][0],colored_edges[dem_lap_lai][(dem-1)][1]) in value]
                total_minflow = 0
                if count_repeat > 1:
                    for i in range(len(positions)):
                        if positions[i] <= dem_lap_lai:
                            total_minflow += list_minflow[positions[i]]
                H.add_edge(colored_edges[dem_lap_lai][(dem-1)][0],colored_edges[dem_lap_lai][(dem-1)][1], weight = f'{total_minflow if count_repeat > 1 else list_minflow[dem_lap_lai]}/{list_lables[dem - 1][0]}/{list_lables[dem - 1][1]}')
                list_copy.append(colored_edges[dem_lap_lai][(dem-1)])
                nx.draw_networkx_edges(G, pos=note_position,ax=ax, edgelist=list_copy, edge_color='red',connectionstyle='arc3,rad=0.1')  # Tô màu cho các cạnh cụ thể                
                labels = nx.get_edge_attributes(G, 'weight')  # Trọng số của các cạnh
                nx.draw_networkx_edge_labels(G, note_position,ax=ax, edge_labels=labels,connectionstyle='arc3,rad=0.1')  # Vẽ trọng số
                labels = nx.get_edge_attributes(H, 'weight')  # Trọng số của các cạnh
                nx.draw_networkx_edge_labels(H, note_position,ax=ax, edge_labels=labels,connectionstyle='arc3,rad=0.1')  # Vẽ trọng số
            dem+= 1 
    else:
        labels = nx.get_edge_attributes(G, 'weight')  # Trọng số của các cạnh
        nx.draw_networkx_edge_labels(G, note_position,ax=ax, edge_labels=labels,connectionstyle='arc3,rad=0.1')  # Vẽ trọng số
    return fig
# Tạo cửa sổ Tkinter
root = tk.Tk()
root.title("GIẢI BÀI TOÁN MIN COST FLOW")
root.geometry("1200x600")  # Đặt kích thước cửa sổ
# Tạo đồ thị NetworkX
G = nx.DiGraph()
# THÊM CÁC ĐỈNH
for i in range(vertex):
    G.add_node(i,pos = note_position[i])
# lấy trọng số
for i in range(len(list_save_edges)):
    G.add_edge(int(list_save_edges[i][0]), int(list_save_edges[i][1]), weight=f'0/{list_save_edges[i][2]}/{list_save_edges[i][3]}') # cần coi
# Biến để xác định trạng thái hiện tại của các cạnh được tô màu đỏ
def nhap_nhay():
    global dem_lap_lai
    global dem
    global H 
    global animation_running
    global toc_do
    # Cho đồ thị dừng lại khi chạy hết đường đi'
   
    if len(list_duong_di) != 0:
        if dem_lap_lai >= len(list_duong_di) - 1 and dem >= len(list_duong_di[dem_lap_lai]):
            dem = 0
            dem_lap_lai = 0
            H.clear()
            animation_running = False
    if animation_running:
        
        # Tạo hình ảnh từ NetworkX đồ thị
        image = create_networkx_image(G)
        # Tạo và hiển thị canvas Tkinter
        canvas = FigureCanvasTkAgg(image, master=root)
        canvas.draw()
        canvas.get_tk_widget().place(x=2,y=2)
        plt.ioff()
        plt.close('all')
        display_result(total_flow, flow, total_cost, vertex)
        if len(list_duong_di) == 0:
                dem = 0
                dem_lap_lai = 0
                H.clear()
                animation_running = False
        root.after(toc_do,nhap_nhay)
        
    
def start_animation(): # bắt đầu chuyển động đồ thị
    global animation_running
    global toc_do
    animation_running = True
    if dem_lap_lai == 0 and dem == 0:
        toc_do = 2000
        text_widget.delete("1.0", tk.END)
    nhap_nhay()
def stop_animation():# làm đồ thị đứng yên 
    global animation_running
    animation_running = False
def speed(): # xem toàn bộ kết quả
    global toc_do
    toc_do = int(toc_do / 2)
def display_result(total_flow, flow, total_cost, vertex):
    global dem
    global dem_lap_lai
    if dem == 1:
        text_widget.insert(tk.END, list_ket_qua[dem_lap_lai][0] + "\n")
        text_widget.insert(tk.END, list_ket_qua[dem_lap_lai][1] + "\n")
        text_widget.insert(tk.END, list_ket_qua[dem_lap_lai][2] + "\n")
        text_widget.insert(tk.END, list_ket_qua[dem_lap_lai][3] + "\n")
    if (dem_lap_lai >= len(list_duong_di) - 1 and dem == 1) or (len(list_duong_di) == 0):
        if total_flow == 0:
            result_text = 'Không thể tìm đường để vận chuyển từ {} đến {}'.format(0, vertex-1)
        elif total_flow == flow:
            result_text = 'Có thể chuyển hết hàng từ {} đến {}'.format(0, vertex-1)
        else:
            result_text = 'Có thể chuyển 1 phần hàng từ {} đến {} và còn {} đơn vị hàng chưa chuyển'.format(0, vertex-1, flow-total_flow)
        total_flow_text = "Total transported flow: {}".format(total_flow)
        total_cost_text = "Total Cost: {}".format(total_cost)
    
        # Xóa nội dung cũ và chèn văn bản mới vào Text widget
        text_widget.insert(tk.END, "\n" + result_text + "\n")
        text_widget.insert(tk.END, total_flow_text + "\n")
        text_widget.insert(tk.END, total_cost_text + "\n")
# Tạo nút 
text_widget = tk.Text(root, height=20, width=50, font=("Helvetica", 12))
text_widget.place(x=700, y=50)
start_button = tk.Button(root, text="Bắt đầu", command=start_animation)
start_button.place(x=50, y=500, width=100, height=30)
stop_button = tk.Button(root, text="Dừng lại", command=stop_animation)
stop_button.place(x=200, y=500, width=100, height=30)
speed_up_button = tk.Button(root, text="Tăng tốc", command=speed)
speed_up_button.place(x=350, y=500, width=100, height=30)
# Chạy ứng dụng
root.mainloop()